In [183]:
from __future__ import division, print_function # Imports from __future__ since we're running Python 2
import os
import numpy as np
import collections as cls
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
import itertools
import math
import glob
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_score
from matplotlib import pylab
from pylab import *
random_state = 10

In [184]:
#get all city names
all_cities_paths =  glob.glob(os.path.join(os.getcwd(), 'data','*'))
cities = []
for city in all_cities_paths:
    cities.append(city.split("/")[-1])
print(cities)
                    

['amsterdam', 'berlin', 'copenhagen', 'dublin', 'edinburgh', 'helsinki', 'lisbon', 'london', 'madrid', 'moscow', 'reykjavik', 'rome', 'tallin', 'vienna', 'paris']


In [185]:
#finds the not common elements of the two lists
#input: list1,list2 --> the lists to be compared
#output: the not common elements
def find_not_common_columns(list1,list2):
    not_common_cols = []
    both_lists = list1 + list2
    for col in set(both_lists):
        if col not in set(list1) or col not in set(list2):
            not_common_cols.append(col)
    return not_common_cols



In [186]:
#loading each city's dataset to a dictionary and stripping their column titles from unwanted characters
#moreover checking which columns are not common for all csv files of a particular city in order to be discarded later
#in addition their last_modified column values are stripped
datasets = dict()
for city in cities:
    city_data = pd.DataFrame() #initializing city's dataframe
    data_path = glob.glob(os.path.join(os.getcwd(), 'data',city,city,city,'*.csv'))
    print('\n\n\n{}\n'.format(city))
    first = True
    file_columns = [] #containing the columns that the first file of the particular city contains
    for f in data_path:
        file_data =pd.read_csv(f,delimiter=",",lineterminator="\n") 
        old_columns = list(file_data.columns.values)
        new_columns = [] #containing the columns after the stripping
        for col in old_columns:
            new_columns.append(col.strip())
        if first:
            file_columns = new_columns
            first = False
        else:
            if file_columns == new_columns:
                print('no difference in the columns')
            else:
                print('DIFFERENCE IN THE COLUMNS REPORTED')
                print('NOT COMMON COLUMNS: {}'.format(find_not_common_columns(file_columns,new_columns)))      
        file_data.columns = new_columns
        file_data['last_modified'] = file_data['last_modified'].apply(lambda x: x.strip()) #stripping all the values of 'last_modified' column
        city_data = pd.concat([city_data,file_data],ignore_index=True)
    print('\n{}\'s dataset length: {}'.format(city,len(city_data.index)))
    #city_data.fillna("(unknown)",inplace=True)
    datasets[city] = city_data




amsterdam

no difference in the columns
no difference in the columns
no difference in the columns
no difference in the columns
no difference in the columns
no difference in the columns
no difference in the columns
no difference in the columns
no difference in the columns
no difference in the columns
no difference in the columns
DIFFERENCE IN THE COLUMNS REPORTED
NOT COMMON COLUMNS: ['survey_id', 'bathrooms', 'city', 'country', 'location', 'name']
DIFFERENCE IN THE COLUMNS REPORTED
NOT COMMON COLUMNS: ['survey_id', 'bathrooms', 'city', 'country', 'location', 'name']

amsterdam's dataset length: 184551



berlin

no difference in the columns
no difference in the columns
no difference in the columns
no difference in the columns
no difference in the columns
no difference in the columns
no difference in the columns
no difference in the columns
no difference in the columns
no difference in the columns
no difference in the columns
no difference in the columns
no difference in the columns
D

In [188]:
#find out which of the columns of each city's dataset have missing values 
#and delete the columns that have only missing values
city_columns = dict() #it is going to conatin the feature list of each city
for city in datasets.keys():
    city_data = datasets[city]
    all_data = len(city_data.index)
    columns = city_data.columns.values
    city_columns[city] = list(columns)
    some_mis = dict()
    only_mis = []
    for col in columns:
        #bools = data[col].to_frame().isin({col: ['(unknown)']}) 
        bools = city_data[col].to_frame().isnull()
        num = len(bools[bools[col] == True].index) #number of mssing values in the particular column
        if num > 0: #if there is at least one missing value
            if num < len(city_data[col].index): #if not all values are missing
                some_mis[col] = num
            else:
                only_mis.append(col) #if all values of the column are missing
    print_str = '\n{} '.format(city)
    if len(some_mis.keys()) > 0:
        first = True
        for col in some_mis.keys():
            if first:
                first = False
                print_str += '\ncols with some missing values--> {}({}%)'.format(col,100*float(some_mis[col]/all_data))
            else:
                print_str += ',{}({}%)'.format(col,100*float(some_mis[col]/all_data))
    if len(only_mis) > 0:
        print_str += '\ncols with only missing values--> {}'.format(only_mis[0])
        for i in range(1,len(only_mis)):
            print_str += ',{}'.format(only_mis[i])
        city_data.drop(only_mis, axis=1, inplace=True) #deleting columns with only missing data
        datasets[city] = city_data
    print(print_str)
        
    


edinburgh 
cols with some missing values--> overall_satisfaction(13.393982944%),city(69.7384655016%),name(69.7411771086%),price(0.00271160703391%),bedrooms(0.139647762246%),accommodates(1.75712135797%),minstay(72.1260354949%),location(69.7384655016%),survey_id(69.7384655016%),room_type(0.00677901758477%)
cols with only missing values--> bathrooms,borough,country

reykjavik 
cols with some missing values--> city(78.223078474%),price(0.0201617419745%),bedrooms(9.57010685723%),overall_satisfaction(16.4363000963%),minstay(57.7745917247%),location(78.223078474%),survey_id(78.223078474%),room_type(0.0201617419745%)
cols with only missing values--> bathrooms,borough,country

helsinki 
cols with some missing values--> city(64.1407968712%),survey_id(64.1407968712%),name(64.3241261305%),bedrooms(0.0427768271816%),overall_satisfaction(12.1425079443%),minstay(88.1935956979%),location(64.1407968712%),property_type(64.1407968712%)
cols with only missing values--> bathrooms,borough,country

paris 
c

In [189]:
datasets['berlin']

,accommodates,bedrooms,city,host_id,last_modified,latitude,location,longitude,minstay,name,neighborhood,overall_satisfaction,price,reviews,room_id,room_type,survey_id
0,2.0,1.0,NaN,2164.0,2014-05-19 07:34:14.697,52.544251,NaN,13.397491,1.0,NaN,Mitte,4.5,41.0,4,1944,Private room,NaN
1,4.0,1.0,NaN,3718.0,2014-05-19 00:01:50.741,52.534240,NaN,13.418094,2.0,NaN,Pankow,5.0,107.0,57,3176,Entire home/apt,NaN
2,4.0,NaN,NaN,11892.0,2014-05-19 08:39:20.351,52.503725,NaN,13.453809,3.0,NaN,Friedrichshain-Kreuzberg,5.0,98.0,32,6083,Entire home/apt,NaN
3,2.0,1.0,NaN,16149.0,2014-05-19 09:33:10.073,52.511713,NaN,13.454768,3.0,NaN,Friedrichshain-Kreuzberg,5.0,121.0,98,6883,Entire home/apt,NaN
4,2.0,1.0,NaN,17391.0,2014-05-19 07:44:28.073,52.542673,NaN,13.415188,2.0,NaN,Pankow,5.0,57.0,56,7071,Private room,NaN
5,4.0,1.0,NaN,36708.0,2014-05-18 22:12:05.767,52.545326,NaN,13.426076,3.0,NaN,Pankow,4.5,74.0,107,10533,Entire home/apt,NaN
6,2.0,NaN,NaN,42081.0,2014-05-18 23:13:30.774,52.496940,NaN,13.425048,3.0,NaN,Friedrichshain-Kreuzberg,5.0,69.0,221,11339,Entire home/apt,NaN
7,5.0,2.0,NaN,54283.0,2014-05-18 22:27:13.518,52.538098,NaN,13.410343,2.0,NaN,Pankow,5.0,161.0,29,13870,Entire home/apt,NaN
8,4.0,1.0,NaN,64696.0,2014-05-18 21:21:26.976,52.504784,NaN,13.435129,3.0,NaN,Friedrichshain-Kreuzberg,5.0,104.0,21,16644,Entire home/apt,NaN
9,2.0,1.0,NaN,67590.0,2014-05-18 21:16:15.591,52.526924,NaN,13.417938,2.0,NaN,Pankow,5.0,63.0,52,17409,Private room,NaN


In [159]:
datasets['vienna'].sample(5)

,accommodates,bedrooms,borough,city,host_id,last_modified,latitude,location,longitude,minstay,neighborhood,overall_satisfaction,price,reviews,room_id,room_type,survey_id
12966,4.0,1.0,22,NaN,90095005,2017-01-11 05:59:07.038049,48.232837,NaN,16.413297,NaN,2232,5.0,98.0,15,14646046,Entire home/apt,NaN
42012,2.0,1.0,12,Vienna,66844235,2017-05-18 08:22:05.321739,48.165339,0101000020E6100000734C16F71F5130406EE00ED42915...,16.316894,NaN,1210,5.0,21.0,9,17048820,Private room,1284.0
31338,3.0,1.0,3,Vienna,5924508,2017-04-15 04:51:29.732504,48.196503,0101000020E610000066A4DE53396530406F48A3022719...,16.395406,NaN,305,5.0,65.0,33,1076688,Entire home/apt,1125.0
37352,4.0,1.0,7,Vienna,8809805,2017-05-18 11:14:44.688002,48.205478,0101000020E61000004E603AADDB5830401406651A4D1A...,16.347102,NaN,705,4.5,74.0,19,5835467,Entire home/apt,1284.0
37712,4.0,1.0,5,Vienna,27655336,2017-05-18 11:16:18.877517,48.182816,0101000020E61000009E0AB8E7F95B3040C097C2836617...,16.359282,NaN,502,5.0,74.0,5,16122215,Entire home/apt,1284.0


In [160]:
#find out which columns are existent to all of the cities and which are not
all_columns = set() # the union of columns appearing in all of the cities
all_common_columns = set() # the intersection of columns appearing in all of the cities
first = True
for city,columns in city_columns.iteritems():
    columns_set = set(columns) #since all the columns of the same city have a unique name
    for col in columns:
        all_columns.add(col)
    if first == False:
        mortals = [] #containing the columns that are going to be removed from the intersection
        for col in all_common_columns:
            if col not in columns_set:
                mortals.append(col)
        for mor in mortals: 
            all_common_columns.remove(mor)
    else:
        for col in columns:
            all_common_columns.add(col) #initializing the intersection
        first = False

all_common_list = list(all_common_columns) #set to list
if len(all_common_list) > 0:
    print_str = 'Features appearing to every city of the dataset:\n{}'.format(all_common_list[0])
    for i in range(1,len(all_common_list)): #the rest
        print_str += ',{}'.format(all_common_list[i])
    print(print_str)
else:
    print("There are no common features between the different cities")


print_str = "\nFeatures appearing only to some cities of the dataset:\n"
first = True
for col in all_columns:
    if(col not in all_common_columns):
        if first == True:
            print_str += col
            first = False
        else:
            print_str += "," + col

print(print_str)


    

Features appearing to every city of the dataset:
overall_satisfaction,neighborhood,longitude,price,bedrooms,accommodates,minstay,reviews,last_modified,room_id,latitude,host_id,borough,room_type

Features appearing only to some cities of the dataset:
city,survey_id,bathrooms,country,property_type,location,name


In [161]:
df = pd.DataFrame([[np.nan, 2, np.nan, 0], [3, 4, np.nan, 1]],columns=list('ABCD'))
df
#df.dropna(axis=0,subset=['A'])

,A,B,C,D
0,NaN,2,NaN,0
1,3.0,4,NaN,1


In [162]:
#checking if some bad values exist in accommodates column for every city
bad_accommodates_value_found = False
error_cities = [] #containing the cities containing bad values in that column
for city,data in datasets.iteritems():
    print('\n{}:\n'.format(city))
    error_found = False
    bools = data['accommodates'].to_frame().isnull()
    numeric_error_entries = len(data[data['accommodates'] < 1].index) #number of entries with values less than 1
    missing_entries = len(bools[bools['accommodates'] == True].index) #number of entries with missing values
    if numeric_error_entries > 0:
        error_found = True
        print('Numeric error for {} entries ({}% of the city\'s dataset)'.format(numeric_error_entries,
                                                                                100*float(numeric_error_entries/
                                                                                         len(data.index))))
    if missing_entries > 0:
        error_found = True
        print('Missing value for {} entries ({}% of the city\'s dataset)'.format(missing_entries,
                                                                                100*float(missing_entries/
                                                                                         len(data.index))))
    if error_found:
        error_cities.append(city)
        bad_accommodates_value_found = True    

    
#deleting the bad value entries:
print('\n\n')
for city in error_cities:
    city_data = datasets[city]
    old_num = len(city_data.index)
    city_data = city_data[city_data['accommodates'] >= 1]
    city_data = city_data.dropna(axis=0,subset = ['accommodates'])
    new_num = len(city_data.index)
    datasets[city] = city_data
    print('{} entries were deleted from {}'.format(old_num - new_num,city))




edinburgh:

Missing value for 1296 entries (1.75712135797% of the city's dataset)

reykjavik:


helsinki:


paris:

Numeric error for 41 entries (0.00584091346188% of the city's dataset)
Missing value for 7739 entries (1.10250803126% of the city's dataset)

madrid:


moscow:

Missing value for 1370 entries (3.81466837445% of the city's dataset)

dublin:

Missing value for 388 entries (0.617490252248% of the city's dataset)

tallin:


berlin:

Missing value for 9517 entries (3.80114389788% of the city's dataset)

rome:

Missing value for 6127 entries (2.11512902391% of the city's dataset)

london:

Missing value for 12414 entries (2.38170153331% of the city's dataset)

lisbon:

Missing value for 2813 entries (2.33188540354% of the city's dataset)

amsterdam:

Missing value for 1319 entries (0.714707587605% of the city's dataset)

copenhagen:

Missing value for 1252 entries (0.802466366276% of the city's dataset)

vienna:

Missing value for 1056 entries (2.50557585536% of the city's dat

In [163]:
for city in datasets.keys():
    city_data = datasets[city]
    city_data['price_per_accommodated'] = city_data['price'] / city_data['accommodates']
    datasets[city] = city_data

In [164]:
#create new column: price per number of people that can be accommodated

#for city in datasets.keys():
    #print('Created for {}...'.format(city))
    #city_data = datasets[city]
    #for index , row in city_data.iterrows():
        #city_data.set_value(index,'price_per_accommodated',row['price']/row['accommodates'])
    #datasets[city] = city_data

In [165]:
#input: p--> price, quartiles --> quartiles of price_per|_accommodated column,
#k --> tukey test's hyperparameter
#output: whether p is a low outlier, a high outlier or a normal instance
def tukey_test(p,quartiles,k):
    iqr = quartiles[2] - quartiles[0]
    lower_bound = quartiles[0] - k * iqr
    if(lower_bound < 1):
        lower_bound = 1 #not the lower bound of tukey test will be used but a fixed positive lower bound
        
    upper_bound  = quartiles[2] + k * iqr
    low_outliers = 0
    high_outliers = 0
    if p < lower_bound:
        return "low_outlier"
    elif p > upper_bound:
        return "high_outlier"
    return "no" 

In [166]:
#checking if 'last_modified' alone can be used as the primary key
for city,data in datasets.iteritems():
    print(city)
    gb = data.groupby(['last_modified'])
    print(gb['last_modified'].count().sort_values(ascending=False)[0])
#for some cities it cannot be used

edinburgh
1
reykjavik
1
helsinki
1
paris
1
madrid
1
moscow
1
dublin
1
tallin
1
berlin
1
rome
7030
london
1
lisbon
1
amsterdam
1
copenhagen
1
vienna
4611


In [167]:
#checking if 'last_modified' along with 'room_id' can be used as the primary key
for city,data in datasets.iteritems():
    print(city)
    print(data.groupby(['last_modified','room_id']).size().sort_values(ascending=False)[0])
#they can be used

edinburgh
1
reykjavik
1
helsinki
1
paris
1
madrid
1
moscow
1
dublin
1
tallin
1
berlin
1
rome
1
london
1
lisbon
1
amsterdam
1
copenhagen
1
vienna
1


In [168]:
df = pd.DataFrame([[1, 2, 2, 0], [3, 4, 3, 1], [3, 4, 3, 1], [3, 4, 3, 1], [3, 4, 3, 1], [3, 4, 3, 1], [3, 4, 3, 1]],columns=list('ABCD'))
df.drop(df.index[[0,2,4]],inplace=True)
df

,A,B,C,D
1,3,4,3,1
3,3,4,3,1
5,3,4,3,1
6,3,4,3,1


In [169]:
#given the fact that some prices might be per month outlier detection method tukey test will be used
#so that these entries will be captures as outliers along with the rest of the outliers the dataset might have
#price_per_accommodated will be used instead of price column

k = 1.5 #tukey test's hyperparameter
all_low_outliers = 0
all_high_outliers = 0
all_data = 0
print('OUTLIER DETECTION AND DELETION')

for city in datasets.keys():
    mortal_indexes = []
    city_data = datasets[city]
    quartiles = []
    quartiles.append(city_data['price_per_accommodated'].to_frame().quantile(q=0.25,axis=0)[0])
    quartiles.append(city_data['price_per_accommodated'].to_frame().mean(axis=0)[0])
    quartiles.append(city_data['price_per_accommodated'].to_frame().quantile(q=0.75,axis=0)[0])
    city_low_outliers = 0
    city_high_outliers = 0
    city_data.reset_index(inplace=True)
    for index,row in city_data.iterrows():
        res = tukey_test(row['price_per_accommodated'],quartiles,k)
        if res == 'low_outlier':
            city_low_outliers += 1
            mortal_indexes.append(index)
            #mortal_room_id.append(row['room_id'])
            #mortal_last_modified.append(row['last_modified'])
        elif res == 'high_outlier':
            city_high_outliers += 1
            mortal_indexes.append(index)
            #mortal_room_id.append(row['room_id'])
            #mortal_last_modified.append(row['last_modified'])
    all_data += len(city_data.index)
    all_low_outliers += city_low_outliers
    all_high_outliers += city_high_outliers
    city_data = city_data.drop(city_data.index[mortal_indexes]) #deleting the outliers by their index
    
    datasets[city] = city_data
    print('{0}\t(LOW)\t{1}\t(HIGH)\t{2}\t(TOTAL)\t{3}\t{4}% of city\'s dataset'.
          format(city,city_low_outliers,city_high_outliers,city_low_outliers + city_high_outliers
                 ,100 * float(city_low_outliers + city_high_outliers)/len(city_data.index)))

print('\n\n\nALTOGETHER\t(LOW)\t{0}\t(HIGH)\t{1}\t(TOTAL)\t{2}\t{3}% of all cities\' datasets'.
          format(all_low_outliers,all_high_outliers,all_low_outliers + all_high_outliers
                 ,100 * float(all_low_outliers + all_high_outliers)/all_data))
            
            
    
    

OUTLIER DETECTION AND DELETION
edinburgh	(LOW)	1	(HIGH)	3015	(TOTAL)	3016	4.34300525596% of city's dataset
reykjavik	(LOW)	0	(HIGH)	1739	(TOTAL)	1739	4.05361305361% of city's dataset
helsinki	(LOW)	0	(HIGH)	900	(TOTAL)	900	5.81996896017% of city's dataset
paris	(LOW)	5	(HIGH)	34508	(TOTAL)	34513	5.23200111574% of city's dataset
madrid	(LOW)	1	(HIGH)	1037	(TOTAL)	1038	4.77022058824% of city's dataset
moscow	(LOW)	340	(HIGH)	2464	(TOTAL)	2804	8.83427851292% of city's dataset
dublin	(LOW)	0	(HIGH)	3450	(TOTAL)	3450	5.84775497059% of city's dataset
tallin	(LOW)	15	(HIGH)	461	(TOTAL)	476	4.60392687881% of city's dataset
berlin	(LOW)	3	(HIGH)	10088	(TOTAL)	10091	4.37286578496% of city's dataset
rome	(LOW)	0	(HIGH)	13726	(TOTAL)	13726	5.08705739339% of city's dataset
london	(LOW)	3	(HIGH)	24110	(TOTAL)	24113	4.974860583% of city's dataset
lisbon	(LOW)	23	(HIGH)	6975	(TOTAL)	6998	6.31468764945% of city's dataset
amsterdam	(LOW)	0	(HIGH)	5727	(TOTAL)	5727	3.22638798907% of city's dataset
copenh